In [1]:
import numpy as np
import matplotlib.pylab as plt

# 4. 신경망 학습

<code>학습</code>: 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것

## 4.2 손실 함수

### 4.2.1 평균 제곱 오차

<code>평균 제곱 오차(mean squared error, MSE)</code>

$$ E = \frac{1}{n} \sum_{k}(y_k - t_k)^2 $$

각 원소의 출력(추정 값)과 정답 레이블(참 값)의 차를 제곱하고 모두 합한 후 평균  

※ 책에는 $\frac{1}{n}$이 아니라 $\frac{1}{2}$로 나와있는데 왜? <span style='color:pink'>!!!질문!!!</span>

$y_k$: 신경망의 출력(신경망이 추정한 값)  
$t_k$: 정답 레이블  
$k$: 데이터의 차원 수

In [21]:
def mean_squared_error(y, t):
    return np.sum((y-t)**2) / len(t)

### 4.2.2 교차 엔트로피 오차

<code>교차 엔트로피 오차(cross entropy error, CEE)</code>

$$ E = -\sum_{k} t_{k} \log{y_{k}} $$

여기서 $\log$는 밑이 $e$인 자연로그  

$y_k$: 신경망의 출력  
$t_k$: 정답 레이블  

$t_k$는 정답에 해당하는 인덱스의 원소만 1이고 나머지는 0 (원핫인코딩)  
→ $t_k=0$일 때는 모두 무시 가능하고, $t_k=1$일 때(정답일 때)만 자연로그 계산

예를 들어 정답 레이블은 '2'가 정답이라 하고 이때의 신경망 출력 $y_k$가 $0.6$이라면 교차 엔트로피 오차는 $-\log0.6 = 0.51$  
같은 조건에서 신경망 출력 $y_k$가 $0.1$이라면 교차 엔트로피 오차는 $-\log0.1 = 2.3$  
(상대적으로 제대로 예측한 경우(신경망 출력이 $0.6$인 경우) loss가 작고  
 상대적으로 잘못 예측한 경우(신경망 출력이 $0.1$인 경우) loss가 큼)

<img src="https://user-images.githubusercontent.com/77653353/192322965-5d57ab8b-a5b1-4b2f-a79c-c9a0cae2b55d.png">

In [22]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

아주 작은 값인 delta를 더하여 절대 0이 되지 않도록, 즉 마이너스 무한대가 발생하지 않도록 한 것

### 4.2.3 미니배치 학습

이제 데이터 하나가 아닌 훈련 데이터 모두에 대한 손실 함수의 합을 구하는 방법을 생각해보자

훈련 데이터 모두에 대한 교차 엔트로피 오차

$$ E = - \frac{1}{N}\sum_{n}\sum_{k}t_{nk}\log y_{nk} $$

데이터가 $N$개라면 $t_{nk}$는 $n$번째 데이터의 $k$차원 째의 값을 의미  

$y_{nk}$: 신경망의 출력  
$t_{nk}$: 정답 레이블  

수식이 좀 복잡해 보이지만 데이터 하나에 대한 손실함수를 단순히 $N$개의 데이터로 확장했을 뿐임  
마지막에 $N$으로 나누어 정규화하여 '평균 손실 함수'를 구하는 것

수백만, 수천만개의 데이터를 일일이 계산하기에는 쉽지 않기 때문에  
훈련 데이터로부터 일부만 골라 학습을 수행하는데 이 일부를 <code>미니배치(mini-batch)</code>

### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

<span style='color:pink'>!!!질문!!!</span> ?

정답 레이블이 원-핫 인코딩인 경우

In [10]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

정답 레이블이 원-핫 인코딩이 아니라 '$2$'나 '$7$' 등의 숫자 레이블로 주어지는 경우

In [1]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y[np.arange(batch_size), t])) / batch_size

정답 레이블이 원-핫 인코딩인 경우, t가 0인 원소는 교차 엔트로피 오차도 0이므로 그 계산을 무시할 수 있음  
  
정답 레이블이 숫자 레이블로 주어지는 경우, np.log(y[np.arange(batch_szie), t])  
np.arange(batch_size)는 0부터 batch_size-1까지 배열을 생성함  
(ex. batch_size=5라면, [0,1,2,3,4]라는 넘파이 배열 생성, t에는 레이블이 [2,7,0,9,4]와 같이 저장됨)  

### 4.2.5 왜 손실 함수를 설정하는가?

우리의 궁극적인 목적은 높은 '정확도'를 끌어내는 매개변수 값을 찾는 것!  
그렇다면 왜 '정확도'라는 지표를 두고 손실 함수의 값을 거쳐갈까?

신경망 학습에서의 <code>미분</code>의 역할에 주목!  
가령 가상의 신경망이 있고, 그 신경망의 어느 한 가중치 매개변수에 주목한다고 해보자  
이때 그 가중치 매개변수의 손실 함수의 미분이란 '가중치 매개변수의 값을 아주 조금 변화시켰을 때, 손실 함수가 어떻게 변하나'라는 의미  
만약 이 미분 값이 음수면, 그 가중치 매개변수를 양의 방향으로 변화시켜 손실 함수의 값을 줄일 수 있고  
만약 이 미분 값이 양수면, 그 가중치 매개변수를 음의 방향으로 변화시켜 손실 함수의 값을 줄일 수 있음  
만약 이 미분 값이 0이면, 가중치 매개변수를 어느 쪽으로 움직여도 손실 함수의 값은 달라지지 않음

계단 함수의 미분은 대부분의 장소(0 이외의 곳)에서 0 → 계단 함수를 활성화 함수로 이용하면 손실함수를 지표로 삼는 게 아무 의미 없어짐  
매개변수의 작은 변화가 주는 파장을 계단 함수가 말살하여 손실 함수의 값에는 아무런 변화가 나타나지 않기 때문

## 4.3 수치 미분

### 4.3.1 미분

<code>미분</code>: 특정 순간의 변화량  
$x$의 작은 변화가 함수$f(x)$를 얼마나 변화시키느냐를 의미

$$ \frac{df(x)}{dx} = \lim_{h→0}\frac{f(x+h)-f(x)}{h} $$

위의 식을 참고하여 함수의 미분을 구하는 계산을 구현해보면

In [17]:
def numerical_diff(f, x):
    h = 10e-50
    return (f(x+h) - f(x)) / h

위의 방식은 두 가지 문제가 있는데
- 반올림 오차 문제 → 미세한 값 $h$를 $10^{-4}$정도로 조절
- 진정한 미분은 $x$위치의 함수의 기울기(접선)이지만 $h$를 무한히 0으로 좁히는 것 불가 → 중심 차분 또는 중앙 차분 사용($(x+h)$와 $(x-h)$일 때의 함수 $f$의 차분 계산)

In [18]:
def numerical_diff(f, x):
    h = 1e-4 # 0.0001
    return (f(x+h) - f(x-h)) / (2*h)

<code>수치 미분(numerical differentiation)</code>: 아주 작은 차분(임의 두 점에서의 함수 값들의 차이)으로 미분을 구하는 것  
<code>해석적 미분(analytic differentiation)</code>: 수식을 전개해 미분을 구하는 것 ex.$y = x^2$의 미분은 $\frac{dy}{dx} = 2x$

### 4.3.3 편미분

$$ f(x_0, x_1) = x_0^2 + x_1^2 $$

In [ ]:
def function_2(x):
    return x[0]**2 + x[1]**2

위와 같은 함수가 있다고 해보자  
인수들의 제곱 합을 계산하는 단순한 식이지만, 이번엔 변수가 2개인 함수

<img width="473" alt="fig 4-8" src="https://user-images.githubusercontent.com/77653353/193848270-9e48b89b-d329-4663-9797-3095f4e597f9.png">

<code>편미분</code>: 변수가 여럿인 함수에 대한 미분, $\frac{\partial{f}}{\partial{x_0}}$이나 $\frac{\partial{f}}{\partial{x_1}}$

## 4.4 기울기

앞에서는 $x_0$와 $x_1$의 편미분을 변수별로 따로 계산했음  
그렇다면 $x_0$와 $x_1$의 편미분을 동시에 계산하려면?

$x_0=3$, $x_1=4$일 때 $(x_0, x_1)$ 양쪽의 편미분을 묶어서 $(\frac{\partial{f}}{\partial{x_0}}, \frac{\partial{f}}{\partial{x_1}})$처럼  
모든 변수의 편미분을 벡터로 정리한 것을 <code>기울기(gradient)</code>라고 함

In [19]:
def numerical_gradient(f, x):               # [3, 4]
    h = 1e-4
    grad = np.zeros_like(x)                 # [0, 0]

    for idx in range(x.size):               # idx=0, x.size=2
        tmp_val = x[idx]                    # tmp_val = 3

        # f(x+h) 계산
        x[idx] = tmp_val + h                # x[0] = 3.0001
        fxh1 = f(x)                         # fxh1 = 25.00060001

        # f(x-h) 계산
        x[idx] = tmp_val - h                # x[0] = 2.9999
        fxh2 = f(x)                         # fxh2 = 24.99940001
        
        grad[idx] = (fxh1 - fxh2) / (2*h)   # grad[0] = 0.0012 / 0.0002 = 6
        x[idx] = tmp_val # 값 복원          # x[0] = 3

    return grad

In [20]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [24]:
numerical_gradient(function_2, np.array([3.0, 4.0]))

array([6., 8.])

In [22]:
numerical_gradient(function_2, np.array([0.0, 2.0]))

array([0., 4.])

In [25]:
numerical_gradient(function_2, np.array([3.0, 0.0]))

array([6., 0.])

기울기는 함수의 '가장 낮은 장소(최솟값)'를 가리키는 것  
기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 줄이는 방향

### 4.4.1 경사법(경사 하강법)

우리의 목표는 학습 단계에서 최적의 매개변수를 찾는 것이고, 이는 손실 함수가 최솟값이 될 때의 매개변수 값  
하지만 광대한 공간 속에서 어느 곳이 최솟값인지 알아내기 쉽지 않음  
이러한 상황에서 기울기를 잘 이용해 함수의 최솟값(또는 가능한 한 작은 값)을 찾으려는 것이 경사법

함수가 최솟값, 극솟값, 안장점이 되는 장소에서는 기울기가 0  
- 극솟값: 국소적인 최솟값  
- 안장점(saddle point): 어느 방향에서 보면 극댓값이고, 다른 방향에서 보면 극솟값이 되는 점

기울어진 방향이 꼭 최솟값을 가리키는 것은 아니지만, 그 방향으로 가야 함수의 값을 줄일 수 있음  
그래서 최솟값이 되는 장소를 찾는 문제에서는 기울기 정보를 단서로 나아갈 방향을 정함

<code>경사법(gradient method)</code>, <code>경사 하강법(gradient descent method)</code>:  
현 위치에서 기울어진 방향으로 일정 거리만큼 이동하고, 이동한 곳에서도 기울기를 구하고, 또 기울어진 방향으로 나아가서 함수의 값을 점차 줄이는 것

수식으로 나타내면

$$ x_0 = x_0 - \eta \frac{\partial{f}}{\partial{x_0}} $$  
$$ x_1 = x_1 - \eta \frac{\partial{f}}{\partial{x_1}} $$

<code>학습률(learning rate)</code>: 매개변수 값을 얼마나 갱신하느냐를 정하는 것, 여기서는 $\eta$

경사 하강법 구현

In [26]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):     # f: 최적화하려는 함수
    x = init_x                                              # init_x: 초깃값
                                                            # lr: 학습률
    for i in range(step_num):                               # step_num: 경사법에 따른 반복 횟수
        grad = numerical_gradient(f, x)
        x -= lr * grad
    
    return x

<code>하이퍼파라미터(hyper parameter)</code>: 사람이 직접 설정해야 하는 매개변수 ex.학습률  
<code>파라미터(parameter)</code>: 훈련 데이터와 학습 알고리즘에 의해서 '자동'으로 획득되는 매개변수 ex.가중치, 편향

## 4.5 학습 알고리즘 구현하기

- 전제  
  신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 함. 신경망 학습은 다음과 같이 4단계로 수행함  
  
- 1단계 - 미니배치  
  훈련 데이터 중 일부를 무작위로 가져옴. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것을 목표로 함  

- 2단계 - 기울기 산출  
  미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시  

- 3단계 - 매개변수 갱신  
  가중치 매개변수를 기울기 방향으로 아주 조금 갱신함  

- 4단계 - 반복  
  1~3단계를 반복함

<code>확률적 경사 하강법(Stochastic Gradient Descent, SGD)</code>: 확률적으로 무작위로 골라낸 데이터에 대해 수행하는 경사 하강법

### 4.5.1 2층 신경망 클래스 구현하기

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.functions import *
from common.gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

numerical_gradient 메서드는 수치 미분 방식으로 각 매개변수의 손실 함수에 대한 기울기를 계산  
gradient 메서드는 오차역전파법을 사용하여 기울기를 계산 (다음 장에서 진행)

### 4.5.2 미니배치 학습 구현하기

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수를 적절히 설정한다.
train_size = x_train.shape[0]
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


### 4.5.3 시험 데이터로 평가하기

위의 코드에서 1에포크별로 훈련 데이터와 시험 데이터에 대한 정확도를 기록함  
<code>에포크(epoch)</code>: 학습에서 훈련 데이터를 모두 소진했을 때의 횟수